In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt

import camb
from camb import model
print(f'Using CAMB {camb.__version__} installed at {os.path.dirname(camb.__file__)}')

Using CAMB 1.5.8 installed at /home/gilee/CAMB/camb


In [16]:
def initialize_camb_params(cosmo_params, z):
    
    h = cosmo_params['h']
    Omega_b = cosmo_params['Omega_b']
    Omega_c = cosmo_params['Omega_c']
    Omega_k = cosmo_params['Omega_k']
    w0 = cosmo_params['w0']
    wa = cosmo_params['wa']
    As = cosmo_params['As']
    ns = cosmo_params['ns']
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h, ombh2=Omega_b*h*h, omch2=Omega_c*h*h, omk=Omega_k)
    pars.DarkEnergy = camb.dark_energy.DarkEnergyPPF(w=w0, wa=wa)
    pars.InitPower.set_params(As=As, ns=ns)
    pars.set_matter_power(redshifts=z, kmax=kmax, k_per_logint=k_per_logint)
    pars.NonLinear = model.NonLinear_none
    
    return pars

In [17]:
def adjust_As_at_z0(pars, cosmo_params, tolerance=1e-20, max_iterations=1000):
    
    z0 = pars.Transfer.PK_redshifts[0]
    
    if z0 != 0:
        print(f"Redshift should be 0 here.")
        return None
    
    iteration = 0
    
    results = camb.get_results(pars)
    sigma8 = results.get_sigma8()
    target_sigma8 = cosmo_params['sigma8_z0_WMAP5']
    
    print(f"Target sigma8 = {target_sigma8} at z = {z0} (tolerance = {tolerance})")
    print(f"Iteration {iteration}: sigma8 = {sigma8[0]}, diff={target_sigma8 - sigma8[0]}, As = {pars.InitPower.As} (trial)")
    
    while (np.abs(target_sigma8 - sigma8[0]) > tolerance) and (iteration < max_iterations):
        
        iteration += 1
        As = pars.InitPower.As * (target_sigma8 / sigma8[0])**2 # new normalization
        pars.InitPower.set_params(As=As, ns=pars.InitPower.ns)
        
        # Get new results and sigma8 value
        results = camb.get_results(pars)
        sigma8 = results.get_sigma8()
        print(f"Iteration {iteration}: sigma8 = {sigma8[0]}, diff={target_sigma8 - sigma8[0]}, As = {pars.InitPower.As}")
        
        
    if iteration >= max_iterations:
        print("Warning: Maximum iterations reached. The tolerance may be too strict.")
    else:
        print("Converged successfully!")
        cosmo_params['As'] = As
        print("As updated in cosmo_params.")
            
    return pars

In [18]:
bias8_z0_WMAP5  = 1.26             # target value
sigma8_z0_WMAP5 = 1/bias8_z0_WMAP5 # target value # sigma8 at z=0

As0       = 2e-9                   # trial value # power spectrum noramlization factor at k=k0
ns        = 0.96                   # power spectral index
h         = 0.72                   # little h, dimensionless hubble parameter, H0 = 100*h [km/s/Mpc]

Omega_m   = 0.26                   # physical total matter density
Omega_b   = 0.044                  # physical baryon density
Omega_c   = Omega_m - Omega_b      # physical CDM (cold dark mattery) density
Omega_l   = 1 - Omega_m            # DE (dark energy) density
Omega_k   = 0.0                    # spatial curvature, omk = 1 - omt

# EoS of DE
w0 = -1
wa = 0

In [19]:
# Cosmological Parameters (WMAP5)
cosmo_params = {
    'As': As0, # trial number
    'ns': ns,
    'h': h,
    'bias8_z0_WMAP5': bias8_z0_WMAP5,
    'Omega_m': Omega_m,
    'Omega_b': Omega_b,
    'Omega_k': Omega_k,
    'w0': w0,
    'wa': wa
}

cosmo_params['sigma8_z0_WMAP5'] = 1/cosmo_params['bias8_z0_WMAP5']
cosmo_params['Omega_c'] = cosmo_params['Omega_m'] - cosmo_params['Omega_b']
cosmo_params['Omega_l'] = 1 - cosmo_params['Omega_m']

In [20]:
print("\nWMAP5 Parameters:")
print("  bias8 = ", bias8_z0_WMAP5)
print("  sigma8 = ", sigma8_z0_WMAP5)
print("  Omega_m = ", Omega_m)
print("  Omega_b = ", Omega_b)
print("  Omega_c = ", Omega_c)
print("  Omega_l = ", Omega_l)
print("  Omega_k = ", Omega_k)
print("  w0 = ", w0)
print("  wa = ", wa)


WMAP5 Parameters:
  bias8 =  1.26
  sigma8 =  0.7936507936507936
  Omega_m =  0.26
  Omega_b =  0.044
  Omega_c =  0.21600000000000003
  Omega_l =  0.74
  Omega_k =  0.0
  w0 =  -1
  wa =  0


In [21]:
# starting redshift
z_start = 200
a_start = 1/(z_start+1)

# simulation box setups
levelmin = 7
levelmax = levelmin + 7
n_cell0  = 2**levelmin
c_Lbox   = 2**levelmax / 1000 # [cMpc]
c_dx0    = c_Lbox / n_cell0   # [cMpc]

print("\nSimulation Box Setups:")
print("  z_start       = ", z_start)
print("  a_start       = ", a_start)

print("\n  Lbox [cMpc]   = ", c_Lbox)
print("  Lbox [cMpc/h] = ", c_Lbox*h)
print("  dx_ini [ckpc] = ", c_dx0 * 1000)
print("  dx_ini [pkpc] = ", c_dx0 * 1000 / (z_start + 1))
print("  dx_fin [pkpc] = ", c_dx0 * 1000 / 2**7)
print("  levelmin      = ", levelmin)
print("  levelmax      = ", levelmax)
print("  Ncell_ini     = ", n_cell0)

# power spectrum params
kmin = 2 * np.pi / c_Lbox       # [1/Mpc] # largest scale (box size)
kmax = np.pi * n_cell0 / c_Lbox # [1/Mpc] # smallest scale (Nyquist wavenumber)
khmin = kmin/h                  # [1/Mpc] # largest scale (box size)
khmax = kmax/h                  # [1/Mpc] # smallest scale (Nyquist wavenumber)
k0  = 0.05                      # [1/Mpc] pivot_scalar
kh0 = k0/h                      # [1/Mpc] pivot_scalar

k_per_logint = 0 # default: 0 # affects the number of k values

print("\nPower Sepctrum Parameters:")
print("\n  kmin   [1/Mpc] = ", kmin)
print("  kmin/h [1/Mpc] = ", khmin)
print("  kmax   [1/Mpc] = ", kmax)
print("  kmax/h [1/Mpc] = ", khmax)


Simulation Box Setups:
  z_start       =  200
  a_start       =  0.004975124378109453

  Lbox [cMpc]   =  16.384
  Lbox [cMpc/h] =  11.796479999999999
  dx_ini [ckpc] =  128.0
  dx_ini [pkpc] =  0.6368159203980099
  dx_fin [pkpc] =  1.0
  levelmin      =  7
  levelmax      =  14
  Ncell_ini     =  128

Power Sepctrum Parameters:

  kmin   [1/Mpc] =  0.38349519697141027
  kmin/h [1/Mpc] =  0.5326322180158476
  kmax   [1/Mpc] =  24.543692606170257
  kmax/h [1/Mpc] =  34.088461953014246


In [22]:
pars   = initialize_camb_params(cosmo_params, z=[0]) # initialize
pars   = adjust_As_at_z0(pars, cosmo_params) # normalization

Target sigma8 = 0.7936507936507936 at z = 0.0 (tolerance = 1e-20)
Iteration 0: sigma8 = 0.7628693474928043, diff=0.03078144615798928, As = 2e-09 (trial)
Iteration 1: sigma8 = 0.7936507936507937, diff=-1.1102230246251565e-16, As = 2.1646544338903376e-09
Iteration 2: sigma8 = 0.7936507936507937, diff=-1.1102230246251565e-16, As = 2.164654433890337e-09
Iteration 3: sigma8 = 0.7936507936507936, diff=0.0, As = 2.1646544338903368e-09
Converged successfully!
As updated in cosmo_params.


In [23]:
As = cosmo_params['As']

In [24]:
pars = initialize_camb_params(cosmo_params, z=[z_start])
results = camb.get_results(pars)

In [25]:
results.get_sigma8()
# get_sigma8(): 0.00548385 for 20xkmax
# get_sigma8(): 0.00548385 for 10xkmax
# get_sigma8(): 0.00548385 for kmax

array([0.00548385])

In [12]:
transfers = results.get_matter_transfer_data()
T = transfers.transfer_data

In [13]:
plus_w0 = '+' if w0 >= 0 else ''
plus_wa = '+' if wa >= 0 else ''
fname = f"camb_transfer_z{z_start:03.0f}_w0{plus_w0}{w0:.1f}_wa{plus_wa}{wa:.1f}_klogint{k_per_logint:03d}_kmax20.txt"
np.savetxt(f"./{fname}", T.reshape(T.shape[:2]).T, fmt="%.10e")
print(f"MUSIC input data is created: ./{fname}")

MUSIC input data is created: ./camb_transfer_z200_w0-1.0_wa+0.0_klogint100_kmax20.txt


In [14]:
print("The number of k values = ", T.shape[1])

The number of k values =  901
